In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import numpy as np
import time
import os
from dotenv import load_dotenv
os.environ["USE_PYGEOS"] = "0"
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
%run own_and_modified_functions.ipynb
warnings.simplefilter(action='ignore')

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [2]:
OUTPUT_DIR = '/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data'
sdata = read_zarr( os.path.join(  OUTPUT_DIR, 'sdata.zarr'  ) )
anndata = sc.read('/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/anndataNucA11') # reset original anndata
# delete anndata
del sdata.table
# create anndata
sdata.table = anndata

In [3]:
n_neighbors = 15
n_PCAs = 23
cluster_resolution = 1
# make umap and do leiden clustering with scanpy functions
make_umap(sdata,n_neighbors=n_neighbors,n_PCAs=n_PCAs)
column_name = 'leiden_'+str(n_PCAs)+'_'+str(n_neighbors)+'_'+str(cluster_resolution)
sc.tl.leiden(sdata.table,resolution=cluster_resolution,key_added=column_name)

In [5]:

# annotate each leiden cluster
# method based on marker genes and similar to 'own_score_genes' but leiden clusters annotated instead of individual cells
# for each leiden cluster, a score is calculated for each cell type: 
# sum of the mean expressions of the markers in leiden cluster - sum of mean expression of the markers in all cells
n_clusters = np.unique(sdata.table.obs[column_name]).size
leiden_mean_expression = {}
for i in range(n_clusters):
    an_cluster = sdata.table[sdata.table.obs[column_name]==str(i)]
    daf = an_cluster.to_df().mean(axis=0)
    pd.DataFrame(daf)
    leiden_mean_expression[i] = daf
leiden_mean_expression[0]


gene
Acta2      -0.114241
Adamtsl2   -0.148784
Adgre1     -0.175207
Adgrg6     -0.054919
Atp6v0d2   -0.183765
              ...   
Vwf        -0.141544
Wnt2        0.783007
Wnt9b      -0.010520
Wt1        -0.053019
Xcr1       -0.021488
Length: 99, dtype: float64

In [10]:
norm_expr_var = True
if norm_expr_var:
    all_mean_expression = sdata.table.to_df().div(sdata.table.to_df().std(axis=0)).mean(axis=0)
    for i in range(n_clusters):
        leiden_mean_expression[i] = leiden_mean_expression[i].div(sdata.table.to_df().std(axis=0))
else:
    all_mean_expression = sdata.table.to_df().mean(axis=0)

In [ ]:
path_marker_genes = "/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/markerGeneListBasic.csv",
marker_genes = pd.read_csv(path_marker_genes[0], sep=',',index_col=0)
scores_leiden_celltype = pd.DataFrame()
cell_types = marker_genes.columns.tolist()
for cell_type in cell_types:
    scores_clusters = []
    for i in range(n_clusters):
        score = 0 
        for gene in marker_genes[marker_genes[cell_type] == 1].index.tolist():
            score = score + (leiden_mean_expression[i][gene] - all_mean_expression[gene])
        scores_clusters.append(score)
    scores_leiden_celltype[cell_type] = scores_clusters


In [ ]:
    # scale scores per cell type (min-max scaler), to make them comparable between cell types 
    # min max scale columns of scores_leiden_celltype
    if norm_cell_type_score:
        scores_leiden_celltype = scores_leiden_celltype.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    # leiden cluster is annotated with the cell type with the highest score
    sc_leiden_cellt = scores_leiden_celltype.idxmax(axis=1).to_dict()
    unknown_cells = scores_leiden_celltype[scores_leiden_celltype.max(axis=1) < 0].index.tolist()
    # change the values of keys in list
    for i in unknown_cells:
        sc_leiden_cellt[i] = 'unknown'
    sc_leiden_cellt = {str(k): v for k, v in sc_leiden_cellt.items()}
    sdata.table.obs["annotation_"+column_name]=sdata.table.obs[column_name] 
    sdata.table.obs["annotation_"+column_name].replace(list(sc_leiden_cellt.keys()),list(sc_leiden_cellt.values()), inplace=True)
    b = pd.DataFrame.from_dict(sc_leiden_cellt, orient='index')
    cell_type_leiden = {}
    cell_types = np.unique(b[0])
    for cell_type in cell_types:
        indices = b.index[b[0] == cell_type].tolist()
        cell_type_leiden[cell_type] = indices
    sdata.table.uns["mapping_cell_type_"+column_name] = cell_type_leiden
    # cleanliness of the annotation based on highest and second highest score
    max_scores = scores_leiden_celltype.max(axis=1)
    second_scores = scores_leiden_celltype.apply(lambda x: x.nlargest(2).values[-1], axis=1)
    cleanliness = (max_scores - second_scores) / ((abs(max_scores) + abs(second_scores)) / 2)
    sc_leiden_cleanl = cleanliness.to_dict()
    for i in unknown_cells:
        sc_leiden_cleanl[i] = 0
    sc_leiden_cleanl = {str(k): v for k, v in sc_leiden_cleanl.items()}
    sdata.table.obs["cleanliness_"+column_name]=sdata.table.obs[column_name] 
    sdata.table.obs["cleanliness_"+column_name].replace(list(sc_leiden_cleanl.keys()),list(sc_leiden_cleanl.values()), inplace=True)

In [23]:
anndataX_org = sdata.table.to_df()
# add 1 to all values to avoid 0 values
anndataX = anndataX_org + 1
sdata.table.X = anndataX
sdata.table.to_df()

gene,Acta2,Adamtsl2,Adgre1,Adgrg6,Atp6v0d2,Axl,C5ar1,Ccr2,Ccr7,Cd14,...,Timd4,Tmem119,Trem2,Upk3b,Vsig4,Vwf,Wnt2,Wnt9b,Wt1,Xcr1
cells,,,,,,,,,,,,,,,,,,,,,
64,6.885541,6.846452,6.787989,6.86197,6.805610,7.352486,6.834334,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
66,6.885541,6.846452,17.000000,6.86197,6.805610,10.398350,13.520537,6.879072,6.920159,6.972885,...,6.849475,17.000000,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
67,6.885541,7.761151,6.787989,6.86197,6.805610,6.683575,6.834334,6.879072,6.920159,6.972885,...,6.849475,9.234311,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,17.000000,6.932416
68,6.885541,6.846452,6.787989,6.86197,6.805610,6.683575,6.834334,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
69,6.885541,6.846452,6.787989,6.86197,6.805610,6.683575,6.834334,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28929,6.885541,6.846452,6.787989,6.86197,6.805610,6.683575,6.834334,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
28944,6.885541,6.846452,6.787989,6.86197,11.334355,6.683575,6.834334,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
28992,6.885541,6.846452,6.787989,6.86197,6.805610,6.683575,9.209950,6.879072,6.920159,6.972885,...,6.849475,6.847248,6.924513,6.981231,6.770621,6.841347,6.817677,6.924638,6.943119,6.932416
